<a href="https://colab.research.google.com/github/Mahnazshamissa/Python/blob/main/asigmo_ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 4


Per group create a teams-meeting where one person opens the colab notebook and shares the screen (preferably a different person than yesterday)

1.   Change the data source from "frankenstein.txt" to "dracula.txt", 
2.   Train the network for 10 epochs and test it on the 3 example sentences
3.   Write down 3 more example sentences and test the model on these as well
4.   Train the LSTM for 50 more epochs and run the completion again. What do you observe?
4.   Add another LSTM layer to our RNN model
5.   Train the 2-layer LSTM for 10 epochs and see how the sentence completion has changed
6.   Change the "dracula.txt" to "trump.txt", which contains Tweets from Donald Trump from 2019 and 2020.
7.   Train a single-layer LSTM on the Tweet's from Trump. If you run out of memory while preprocessing the data, change ```step = 3``` to ```step = 10```.
8.   Run the completion on the six sentences from before again. What do you observe?



**Question**
The models make very few spelling errors and few grammar error, but the content is mostly nonsense. Why do you think are the error types so different?

We will meet in the general channel at 12:10!

In [ ]:
! nvidia-smi

In [ ]:
import tensorflow as tf
import numpy as np

# Downloading the data

In [ ]:
filename = "frankenstein.txt"
path = tf.keras.utils.get_file(
    filename, origin=f"https://pub.ist.ac.at/~mlechner/datasets/{filename}"
)
with open(path, encoding="utf-8") as f:
    text = f.read().lower()           # Make lowercase 
text = text.replace("\n", " ")        # Remove line-breaks & newlines
print("Corpus length:", len(text))

In [ ]:
# Print the first 100 characters of the file we downloaded
print("First 100 characters: ",text[:100])
print("Last 100 characters : ",text[-100:])

# Define input features and output labels

In [ ]:
chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

In [ ]:
print("First sentence:")
print(sentences[0])
print("First label: ")
print(next_chars[0])

# Transforming text into numerical data

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros(len(sentences), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i] = char_indices[next_chars[i]]

In [ ]:
# First training samples
print(x[0])
print("label: ")
print(y[0])

# Splitting training and validation data

In [ ]:
# We can define our training and validation dataset as simple numpy arrays
split_index = int(0.2*x.shape[0])
valid_x, valid_y = x[-split_index:],y[-split_index:]
train_x, train_y = x[:-split_index],y[:-split_index]
print("valid_x.shape",valid_x.shape)
print("valid_y.shape",valid_y.shape)
print("train_x.shape",train_x.shape)
print("train_y.shape",train_y.shape)

# Defining our Recurrent Neural Network

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(maxlen, len(chars))),
        tf.keras.layers.LSTM(128),
        tf.keras.layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["sparse_categorical_accuracy"])
model.summary()

# Training

In [ ]:
training_log = model.fit(train_x, train_y, batch_size=64, epochs=10, validation_data=(valid_x,valid_y))

# Let's try out our model

In [ ]:
def sample(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    
def complete_sentence(sentence):
  if len(sentence) < maxlen:
    raise ValueError(f"Sentence must have at least {maxlen} characters, but provided sentence had only {sentence}")
  if len(sentence) > maxlen:
    sentence = sentence[-maxlen:]
  generated = ""
  print(f"Continuing sentence '{sentence}' ...")

  for i in range(200):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char

  print("... ", generated)
  print()

In [ ]:
complete_sentence("this is an example to show how the machine would continue")

In [ ]:
### TODO: Add 3 more sentences to complete. Think about what types of sentences would be interesting to test

# Biased data -> biased model

In [ ]:
complete_sentence("    was the thiefs answer, to which the man replied ")
complete_sentence("    was the thiefs answer, to which the woman replied ")